<a href="https://colab.research.google.com/github/nakyeong-kim/python_advanced/blob/main/8_Descriptor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 8. Descriptor

### descriptor, \_\_set__, \_\_get__, \_\_delete__, property  
### descriptor(low level) vs. property(high level)<br><br>

* Descriptor
  1. Descriptor : \_\_get__, \_\_set__ 또는 \_\_delete__ 스페셜 메소드 중 한개 이상 구현 되어 있는 객체
  2. 다른 객체의 속성으로 정의될 수 있음 → 즉, 클래스의 속성을 객체로 매핑해서, 그 객체에서 지정된 메소드로 동작을 하게 하는 것이 Descriptor
  3. 읽기, 쓰기, 삭제 연산을 할 때 동작에 따라 각각 구현된 스페셜 메소드가 호출됨
  4. data descriptor(\_\_set__, \_\_delete__), non-data descriptor(\_\_get__)
  5. 읽기 전용 객체 생성 가능 + 클래스를 의도하는 방향으로 생성 가능
  6. 코드 간결 + 재사용성 증가 + 상황에 맞는 메소드 구현을 통한 객체 지향 프로그래밍 구현 가능
  7. property와 달리 reuse(재사용) 가능
  8. ORM Framework 사용
<br><br>

* 유명한 프레임워크들은 이러한 Descriptor나 Meta Class 형식으로 코딩되어 있음
* 전세계 개발자들이 지속적으로 개선하므로 이러한 것들을 읽을 줄 알아야 함
* 혼자 개발할 때에도 이런 식으로 코딩하면 좀 더 짜임새 있는 코딩 가능


<br><br><br>

In [1]:
# Ex1 - 기본적인 Descriptor

class DescriptorEx1(object):
  def __init__(self, name='Default'):
    self.name = name

  def __get__(self, obj, objtype):
    return 'Get method called. → self : {}, obj : {}, objtype {}, name : {}'.format(self, obj, objtype, self.name)

  def __set__(self, obj, name):
    print('Set method called.')
    if isinstance(name, str):
      self.name = name
    else:
      raise TypeError('Name should be string.')

  def __delete__(self, obj):
    print('Delete method called.')
    self.name = None

class Sample1(object):
  name = DescriptorEx1()


s1 = Sample1()

# 자동으로 맞는 메소드 호출하여 작동

s1.name = 'Descriptor Test1'  # __set__ 호출
# s1.name = 10  # 예외 발생
print('Ex1 > ', s1.name)      # __get__ 호출
del s1.name     # __delete__ 호출
s1.name         # delete 확인

Set method called.
Ex1 >  Get method called. → self : <__main__.DescriptorEx1 object at 0x7bfea63b3a60>, obj : <__main__.Sample1 object at 0x7bfea63b3bb0>, objtype <class '__main__.Sample1'>, name : Descriptor Test1
Delete method called.


"Get method called. → self : <__main__.DescriptorEx1 object at 0x7bfea63b3a60>, obj : <__main__.Sample1 object at 0x7bfea63b3bb0>, objtype <class '__main__.Sample1'>, name : None"

In [2]:
# Ex2 - Property 클래스 사용하여 Descriptor 직접 구현
# class property(fget=None, fset=None, fdel=None, doc=None)
# descriptor를 한 클래스에서 커스터마이징해서 쓸 때에는 원하는 이름으로 쓸 수 있음

class DescriptorEx2(object):
  def __init__(self, value):
    self._name = value

  def getVal(self):
    return 'Get method called. → self : {}, name : {}'.format(self, self._name)

  def setVal(self, value):
    print('Set method called.')
    if isinstance(value, str):
      self._name = value
    else:
      raise TypeError('Name should be string.')

  def delVal(self):
    print('Delete method called.')
    self._name = None

  name = property(getVal, setVal, delVal, 'Property 테스트를 하는 name 필드입니다. 의미는 없습니다.')


s2 = DescriptorEx2('Descriptor Test2')    # Ex2는 Default 안했으니 초기화 넣어줘야 함


print('Ex2 >', s2.name)   # 최초 값 확인  # getVal 호출
s2.name = 'Descriptor Test2 Method.'      # setVal 호출
# s2.name = 10    # 예외 발생
del s2.name       # delVal 호출
print('Ex2 >', s2.name)   # delete 확인
print('Ex2 >', DescriptorEx2.name.__doc__)   # doc 호출   # 내가 만든 소스를 누군가 갖다 쓰려고 할 때 설명서 확인

Ex2 > Get method called. → self : <__main__.DescriptorEx2 object at 0x7bfea63b3d90>, name : Descriptor Test2
Set method called.
Delete method called.
Ex2 > Get method called. → self : <__main__.DescriptorEx2 object at 0x7bfea63b3d90>, name : None
Ex2 > Property 테스트를 하는 name 필드입니다. 의미는 없습니다.


In [3]:
# Ex3 - Descriptor 예제 (1)

import os

class DirectoryFileCount:
  def __get__(self, obj, objtype=None):
    print(os.listdir(obj.dirname))
    return len(os.listdir(obj.dirname))

class DirectoryPath:
  # Descriptor instance
  size = DirectoryFileCount()

  def __init__(self, dirname):
    self.dirname = dirname

# 현재 경로
s = DirectoryPath('./')
# 이전 경로
g = DirectoryPath('../')

print(s.size)
print(g.size)


# 헷갈릴 때 출력 용도
print('Ex3 > ', dir(DirectoryPath))
print('Ex3 > ', DirectoryPath.__dict__)
print('Ex3 > ', dir(s))
print('Ex3 > ', s.__dict__)

['.config', 'sample_data']
2
['etc', 'opt', 'root', 'var', 'srv', 'mnt', 'home', 'media', 'sbin', 'usr', 'dev', 'tmp', 'lib', 'lib32', 'bin', 'lib64', 'proc', 'run', 'libx32', 'sys', 'boot', '.dockerenv', 'tools', 'datalab', 'content', 'python-apt', 'NGC-DL-CONTAINER-LICENSE', 'cuda-keyring_1.0-1_all.deb']
28
Ex3 >  ['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'size']
Ex3 >  {'__module__': '__main__', 'size': <__main__.DirectoryFileCount object at 0x7bfea63faec0>, '__init__': <function DirectoryPath.__init__ at 0x7bfea63cd120>, '__dict__': <attribute '__dict__' of 'DirectoryPath' objects>, '__weakref__': <attribute '__weakref__' of 'DirectoryPath' objects>, '__doc__': None}
Ex3 >  ['__class_

In [17]:
# Ex4 - Descriptor 예제 (2)

import logging

logging.basicConfig(
    format='%(asctime)s %(message)s',
    level=logging.INFO,
    datefmt='%Y-%m-%d %H:%M:%S'
)

class LoggedScoreAccess:
  def __init__(self, value=50):
    self.value = value

  def __get__(self, obj, objtype=None):
    logging.info('Accessing %r giving %r', 'score', self.value)
    return self.value

  def __set__(self, obj, value):
    logging.info('Updating %r to %r', 'score', self.value)
    self.value = value

class Student:
  # Descriptor instance
  score = LoggedScoreAccess()

  def __init__(self, name):
    # Regular instance attribute
    self.name = name


s1 = Student('Kim')
s2 = Student('Lee')

# 점수 확인 : s1
print('Ex4 > ', s1.score)   # colab 환경에서 logging 안찍히는 듯함
s1.score += 20    # s1.score = s1.score + 20  # __get__ 호출 후, __set__ 호출
print('Ex4 > ', s1.score)

# 점수 확인 : s2
print('Ex4 > ', s2.score)
s2.score += 30
print('Ex4 > ', s2.score)

# __dict__ 확인
print('Ex4 > ', vars(s1))
print('Ex4 > ', vars(s2))
print('Ex4 > ', s1.__dict__)
print('Ex4 > ', s2.__dict__)

Ex4 >  50
Ex4 >  70
Ex4 >  70
Ex4 >  100
Ex4 >  {'name': 'Kim'}
Ex4 >  {'name': 'Lee'}
Ex4 >  {'name': 'Kim'}
Ex4 >  {'name': 'Lee'}
